### Información relevante
La diferencia de este script con la versión 1 es que este omite los cancelados sin necesidad de indicarlos, partiendo de la premisa que todo estudiante debe de tener 3 notas registradas en las definitivas. Además, no incluye información relacionada a 100k.

In [1]:
import pandas as pd
from datetime import date

In [2]:
DATOS = pd.read_csv('./INPUT/DEFINITIVAS-C2-1-12.csv', skiprows=1, sep=';', encoding='latin-1')
CODIGOS = DATOS['ESTUDIANTE'].unique()
CICLO = 2
# La llave debe ser el nombre de la asignatura y el contenido una tupla (porcentaje, haceParteParcial),
# donde haceParteParcial es Verdadero si y solo si se requiere en el calculo individual de notas parciales.
# por defecto debe ser Verdadero si la asignatura NO es la asignatura con retos/sprints.
NOMBRE_ASIG_PRINCIPAL = 'JAVA'
ASIGNATURAS = {
    NOMBRE_ASIG_PRINCIPAL: (0.8, False),
    'INGLES II': (0.1, True),
    'COACHING II': (0.1, True)
}
# Necesario por si el ciclo cuenta con más de una asignatura principal, como lo es el ciclo 4
PROGRAMACIONES = ['JAVA']
UMBRAL_REPROBADO_SA = 2.945
UMBRAL_REPROBADO_100K = 3
REDONDEO_DECIMALES_SA = 1
REDONDEO_DECIMALES_100K = 2

INCLUIR_RETOS = False
if INCLUIR_RETOS:
    DATOS_PARCIALES = pd.read_csv('./INPUT/PARCIALES-C1-1-12.csv', skiprows=1, sep=';', encoding='latin-1')
# Los porcentajes de la siguiente lista NO deben ser los de Moodle sino los dados por el MinTIC/Operador para EL CICLO
RETOS = [
    ('RETO_1', 0.1),
    ('RETO_2', 0.1),
    ('RETO_3', 0.2),
    ('RETO_4', 0.2),
    ('RETO_5', 0.2),
]
N_RETOS = len(RETOS)

In [3]:
rta = {
    'CÓDIGO': [],
    'NOMBRE': [],
    'APELLIDO': [],
    'DEFINITIVA_REAL_SA': [],
    'DEFINITIVA_CICLO_SA': [],
    'REPROBÓ_SA': [],
}
for asignatura in ASIGNATURAS:    
    rta[asignatura] = []
if INCLUIR_RETOS:
    for reto, _ in RETOS:
        rta[reto] = []
    rta['DEFINITIVA_100K'] = []
    rta['REPROBÓ_100K'] = []
    rta['SA_IGUAL_A_100K'] = []
    rta['DIFERENCIA_DEFSA_DEF100K'] = []

omitidos = []
for cod in CODIGOS:
    if cod in omitidos:
        continue
    dfDefinitiva = DATOS[DATOS['ESTUDIANTE'] == cod]
    # Omisión de cancelados
    if len(dfDefinitiva) < 3:
        omitidos.append(cod)
        continue
    acum = 0
    acumP = 0
    for index, row in dfDefinitiva.iterrows():
        asignaturaActual = row['ASIGNATURA']
        asignaturaActual = asignaturaActual if asignaturaActual not in PROGRAMACIONES else NOMBRE_ASIG_PRINCIPAL
        rta[asignaturaActual].append(row['NOTA'])
        nota = row['NOTA'] * ASIGNATURAS[asignaturaActual][0]
        acum += nota
        acumP += nota if ASIGNATURAS[asignaturaActual][1] else 0
        
    definitiva = round(acum, REDONDEO_DECIMALES_SA)
    repSA = True if definitiva < UMBRAL_REPROBADO_SA else False
    rta['CÓDIGO'].append(cod)
    rta['NOMBRE'].append(row['NOMBRES'])
    rta['APELLIDO'].append(row['APELLIDOS'])
    rta['DEFINITIVA_REAL_SA'].append(acum)
    rta['DEFINITIVA_CICLO_SA'].append(definitiva)
    rta['REPROBÓ_SA'].append(repSA)
    
    if INCLUIR_RETOS:
        dfParciales = -1
        for asig in PROGRAMACIONES:
            dfTestP = DATOS_PARCIALES[(DATOS_PARCIALES['ESTUDIANTE'] == cod) & (DATOS_PARCIALES['ASIGNATURA'] == asig)]
            if len(dfTestP) != 0:
                dfParciales = dfTestP
                break
        if type(dfParciales) == int or len(dfParciales) != N_RETOS:
            print ('Error en número de actividades parciales', cod)
        for i in range(N_RETOS):
            reto, porcentaje = RETOS[i]
            notaP = dfParciales[dfParciales['No'] == (i+1)]['NOTA'].iloc[0]
            rta[reto].append(notaP)
            acumP += notaP * porcentaje
        def100k = round(acumP, REDONDEO_DECIMALES_100K)
        rep100k = True if def100k < UMBRAL_REPROBADO_100K else False
        rta['DEFINITIVA_100K'].append(def100k)
        rta['REPROBÓ_100K'].append(rep100k)
        rta['SA_IGUAL_A_100K'].append(repSA == rep100k)
        rta['DIFERENCIA_DEFSA_DEF100K'].append(definitiva - def100k)

In [4]:
for llave, valor in rta.items():
    print (llave, len(valor))

CÓDIGO 4681
NOMBRE 4681
APELLIDO 4681
DEFINITIVA_REAL_SA 4681
DEFINITIVA_CICLO_SA 4681
REPROBÓ_SA 4681
JAVA 4681
INGLES II 4681
COACHING II 4681


In [5]:
# Generación y guardado Dataset definitivas
rCompleto = pd.DataFrame(rta)
ordenRCompleto = [
    'CÓDIGO',
    'NOMBRE',
    'APELLIDO',
]
ordenRCompleto += list(ASIGNATURAS.keys())
if INCLUIR_RETOS:
    ordenRCompleto += [reto[0] for reto in RETOS]
ordenRCompleto += ['DEFINITIVA_REAL_SA', 'DEFINITIVA_CICLO_SA', 'REPROBÓ_SA']
if INCLUIR_RETOS:
    ordenRCompleto += ['DEFINITIVA_100K', 'REPROBÓ_100K', 'SA_IGUAL_A_100K', 'DIFERENCIA_DEFSA_DEF100K']
rCompleto = rCompleto[ordenRCompleto]
rCompleto.to_excel('./OUTPUT/DEFINITIVAS_CICLO_' + str(CICLO) + '_' + str(date.today()) + '.xlsx', index = False)